In [7]:
%run "../main_global.ipynb"

Connection with MySQL database is ready!


In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import fastdtw

In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("dtw").getOrCreate()
from pyspark.sql.types import StringType, StructType, StructField, FloatType, IntegerType
from pyspark.sql.functions import pandas_udf, PandasUDFType

In [10]:
target = "pm25"
mvi_method = "MVI_MICE"
sqlq = "Select * from {}_sima_{} where year(datetime) in (2020, 2021, 2022)".format(mvi_method, target)
df = qdata(sqlq).set_index("datetime")

df.head()

,SE,NE,CE,NO,SO,NO2,NTE,NE2,SE2,SO2,SE3,SUR,NTE2,NE3
datetime,,,,,,,,,,,,,,
2020-01-01 00:00:00,54.000,108.0000,61.0,71.0000,59.0000,46.3536,50.0,22.0000,90.4017,22.1644,18.0000,19.9460,62.0000,0.0
2020-01-01 01:00:00,49.965,40.8865,86.0,42.7064,51.0000,5.0000,47.0,23.6457,27.2567,37.0000,28.0000,11.0000,34.5392,0.0
2020-01-01 02:00:00,54.000,45.0000,72.0,83.0000,51.0000,6.0000,44.0,29.4851,36.1422,42.0000,28.0000,20.1893,54.6086,0.0
2020-01-01 03:00:00,36.000,35.8743,70.0,51.0000,48.0000,5.0000,47.0,9.0000,24.6768,36.0000,11.0000,31.0000,38.0000,0.0
2020-01-01 04:00:00,34.000,43.0000,94.0,56.0000,65.5813,5.0000,33.0,16.4763,28.4554,47.0000,24.9822,12.0000,39.5703,0.0


In [ ]:

# Import Data
sqlq = "Select * from {}_sima_{} where year(datetime) in (2020, 2021, 2022)".format(mvi_method, target)
df = qdata(sqlq).set_index("datetime")


# Use schema
df1 = df.withColumn('Sub_metering_1',df.Sub_metering_1.cast('float')) \
    .withColumn('Sub_metering_2',df.Sub_metering_2.cast('float')) \
    .withColumn('Sub_metering_3',df.Sub_metering_3.cast('float')) \
    .withColumn('Voltage',df.Voltage.cast('float'))

df1.show(5)

df2 = df1.groupby("Date").agg(F.collect_list("Sub_metering_1").alias('meter1'),F.collect_list("Sub_metering_2").alias('meter2'),F.collect_list("Sub_metering_3").alias('meter3'))

df2.show(5)

def get_udf_distance(array1, array2):
    distance, path = fastdtw(array1, array2, dist=euclidean)
    return distance

udf_dtw = udf(get_udf_distance , FloatType())
df3 = df2.select('Date', udf_dtw(df2.meter1, df2.meter2).alias('dtw_distance (meter1-meter2)'))

df3.show()